# Lab 8 Parsing

Lab session by:
* Daniel Hess
* Pandelis Laurens Symeonidis

# **NOTES**
chartparser (seen in class), can access other strategies using parameter of chartparser class or use the specific class (bottomup, leftcorner …) 

viterbi parser → will not find all solutions only optimal but will find very quick

order matters

first rule should be whole sentence

we can test different strategies

exercise:

given the sentence, expand (add rules to the grammar in the first slide) so the grammar can parse that sentence (dont add too specific rules)

perform parsing using different strategies

for each one of them print the returned trees, number of edges and list of explored edges.

which parser is the most efficient based on the nr of edges that r explored

which edges are filtered out by each strat and why. use edges method create a set and compare the edges that were explored and explain why.

## Imports

In [1]:
from nltk.parse import BottomUpChartParser, BottomUpLeftCornerChartParser, LeftCornerChartParser
from nltk import CFG
from nltk.parse.util import Chart


# Define grammar

In [2]:
grammar = CFG.fromstring('''
  S   -> NP VP
  VP -> V NP | V PP
  PP -> P NP
  NP  -> NNS | JJ NNS | NP CC NP
  NNS -> "cats" | "dogs" | "mice" | NNS CC NNS 
  JJ  -> "big" | "small" | "lazy"
  CC  -> "and" | "or"
  P -> "with"
  V -> "play" | "sleep"
  ''')
print("Grammar rules:\n")
for prod in grammar.productions():
    print(prod)

Grammar rules:

S -> NP VP
VP -> V NP
VP -> V PP
PP -> P NP
NP -> NNS
NP -> JJ NNS
NP -> NP CC NP
NNS -> 'cats'
NNS -> 'dogs'
NNS -> 'mice'
NNS -> NNS CC NNS
JJ -> 'big'
JJ -> 'small'
JJ -> 'lazy'
CC -> 'and'
CC -> 'or'
P -> 'with'
V -> 'play'
V -> 'sleep'


# Helper function to run a parser

In [3]:
def run_parser(parser_name, grammar, sentence):
    if (parser_name == "BottomUpChartParser"):
        parser_class = BottomUpChartParser
    elif (parser_name == "BottomUpLeftCornerChartParser"):
        parser_class = BottomUpLeftCornerChartParser
    elif (parser_name == "LeftCornerChartParser"):
        parser_class = LeftCornerChartParser

    parser = parser_class(grammar)

    chart = parser.chart_parse(sentence)
    return chart



# Experiment

In [4]:
sentence = "lazy cats play with mice".split()

def analyse(chart: Chart):
    trees = list(chart.parses(grammar.start()))
    print("\nParse trees:")
    if not trees:
        print("  (no complete parse found)")
    else:
        for i, t in enumerate(trees, start=1):
            print(f"\nTree {i}:")
            print(t)

    edges = list(chart.edges())
    print(f"\nNumber of edges: {len(edges)}")

    print("\nExplored edges:")
    for e in edges:
        print(e)


# Bottom Up Chart Parser

In [10]:
parse_bottom_up = run_parser("BottomUpChartParser", grammar, sentence)
analyse(parse_bottom_up)


Parse trees:

Tree 1:
(S
  (NP (JJ lazy) (NNS cats))
  (VP (V play) (PP (P with) (NP (NNS mice)))))

Number of edges: 48

Explored edges:
[0:1] 'lazy'
[1:2] 'cats'
[2:3] 'play'
[3:4] 'with'
[4:5] 'mice'
[0:0] JJ -> * 'lazy'
[0:1] JJ -> 'lazy' *
[0:0] NP -> * JJ NNS
[0:1] NP -> JJ * NNS
[1:1] NNS -> * 'cats'
[1:2] NNS -> 'cats' *
[1:1] NP -> * NNS
[1:1] NNS -> * NNS CC NNS
[0:2] NP -> JJ NNS *
[1:2] NP -> NNS *
[1:2] NNS -> NNS * CC NNS
[1:1] S  -> * NP VP
[1:1] NP -> * NP CC NP
[1:2] S  -> NP * VP
[1:2] NP -> NP * CC NP
[0:0] S  -> * NP VP
[0:0] NP -> * NP CC NP
[0:2] S  -> NP * VP
[0:2] NP -> NP * CC NP
[2:2] V  -> * 'play'
[2:3] V  -> 'play' *
[2:2] VP -> * V NP
[2:2] VP -> * V PP
[2:3] VP -> V * NP
[2:3] VP -> V * PP
[3:3] P  -> * 'with'
[3:4] P  -> 'with' *
[3:3] PP -> * P NP
[3:4] PP -> P * NP
[4:4] NNS -> * 'mice'
[4:5] NNS -> 'mice' *
[4:4] NP -> * NNS
[4:4] NNS -> * NNS CC NNS
[4:5] NP -> NNS *
[4:5] NNS -> NNS * CC NNS
[4:4] S  -> * NP VP
[4:4] NP -> * NP CC NP
[3:5] PP -> P 

# Bottom Up Left Corner Chart Parser

In [11]:
parse_bottom_up_left_corner = run_parser("BottomUpLeftCornerChartParser", grammar, sentence)
analyse(parse_bottom_up_left_corner)


Parse trees:

Tree 1:
(S
  (NP (JJ lazy) (NNS cats))
  (VP (V play) (PP (P with) (NP (NNS mice)))))

Number of edges: 29

Explored edges:
[0:1] 'lazy'
[1:2] 'cats'
[2:3] 'play'
[3:4] 'with'
[4:5] 'mice'
[0:1] JJ -> 'lazy' *
[0:1] NP -> JJ * NNS
[1:2] NNS -> 'cats' *
[1:2] NP -> NNS *
[1:2] NNS -> NNS * CC NNS
[0:2] NP -> JJ NNS *
[0:2] S  -> NP * VP
[0:2] NP -> NP * CC NP
[1:2] S  -> NP * VP
[1:2] NP -> NP * CC NP
[2:3] V  -> 'play' *
[2:3] VP -> V * NP
[2:3] VP -> V * PP
[3:4] P  -> 'with' *
[3:4] PP -> P * NP
[4:5] NNS -> 'mice' *
[4:5] NP -> NNS *
[4:5] NNS -> NNS * CC NNS
[4:5] S  -> NP * VP
[4:5] NP -> NP * CC NP
[3:5] PP -> P NP *
[2:5] VP -> V PP *
[0:5] S  -> NP VP *
[1:5] S  -> NP VP *


# Left Corner Chart Parser

In [ ]:
parse_left_corner = run_parser("LeftCornerChartParser", grammar, sentence)
analyse(parse_left_corner)


Parse trees:

Tree 1:
(S
  (NP (JJ lazy) (NNS cats))
  (VP (V play) (PP (P with) (NP (NNS mice)))))

Number of edges: 22

Explored edges:
[0:1] 'lazy'
[1:2] 'cats'
[2:3] 'play'
[3:4] 'with'
[4:5] 'mice'
[0:1] JJ -> 'lazy' *
[0:1] NP -> JJ * NNS
[1:2] NNS -> 'cats' *
[1:2] NP -> NNS *
[0:2] NP -> JJ NNS *
[0:2] S  -> NP * VP
[1:2] S  -> NP * VP
[2:3] V  -> 'play' *
[2:3] VP -> V * PP
[3:4] P  -> 'with' *
[3:4] PP -> P * NP
[4:5] NNS -> 'mice' *
[4:5] NP -> NNS *
[3:5] PP -> P NP *
[2:5] VP -> V PP *
[0:5] S  -> NP VP *
[1:5] S  -> NP VP *


In [20]:
parsers_edges = {
    "BottomUp": set(parse_bottom_up.edges()),
    "BottomUpLeftCorner": set(parse_bottom_up_left_corner.edges()),
    "LeftCorner": set(parse_left_corner.edges()),
}
names = list(parsers_edges.keys())
for i in range(len(names)):
    for j in range(i + 1, len(names)):
        n1, n2 = names[i], names[j]
        e1, e2 = parsers_edges[n1], parsers_edges[n2]

        only1 = e1 - e2
        only2 = e2 - e1

        print(f"\nEdges only in {n1} (not in {n2}): {len(only1)}")
        for e in sorted(only1, key=str):
            print("  ", e)

        print(f"\nEdges only in {n2} (not in {n1}): {len(only2)}")
        for e in sorted(only2, key=str):
            print("  ", e)



Edges only in BottomUp (not in BottomUpLeftCorner): 19
   [0:0] JJ -> * 'lazy'
   [0:0] NP -> * JJ NNS
   [0:0] NP -> * NP CC NP
   [0:0] S  -> * NP VP
   [1:1] NNS -> * 'cats'
   [1:1] NNS -> * NNS CC NNS
   [1:1] NP -> * NNS
   [1:1] NP -> * NP CC NP
   [1:1] S  -> * NP VP
   [2:2] V  -> * 'play'
   [2:2] VP -> * V NP
   [2:2] VP -> * V PP
   [3:3] P  -> * 'with'
   [3:3] PP -> * P NP
   [4:4] NNS -> * 'mice'
   [4:4] NNS -> * NNS CC NNS
   [4:4] NP -> * NNS
   [4:4] NP -> * NP CC NP
   [4:4] S  -> * NP VP

Edges only in BottomUpLeftCorner (not in BottomUp): 0

Edges only in BottomUp (not in LeftCorner): 26
   [0:0] JJ -> * 'lazy'
   [0:0] NP -> * JJ NNS
   [0:0] NP -> * NP CC NP
   [0:0] S  -> * NP VP
   [0:2] NP -> NP * CC NP
   [1:1] NNS -> * 'cats'
   [1:1] NNS -> * NNS CC NNS
   [1:1] NP -> * NNS
   [1:1] NP -> * NP CC NP
   [1:1] S  -> * NP VP
   [1:2] NNS -> NNS * CC NNS
   [1:2] NP -> NP * CC NP
   [2:2] V  -> * 'play'
   [2:2] VP -> * V NP
   [2:2] VP -> * V PP
   [2:3] VP 

# Analysis

## Chart Parsing Strategies: Efficiency and Filtering Analysis

### Overview
This exercise was to create a set of grammar rules capable of parsing the sentence:
**lazy cats play with mice**

Once the grammar includes adjectives, plural nouns, the verb play, and prepositional phrases, the sentence is parsed using three strategies

* **BottomUpChartParser**
* **BottomUpLeftCornerChartParser**
* **LeftCornerChartParser**

After this we examined the resulting parse tree, along with the number of edges explored and the specific edges that were generated during parsing. 

### Results summary

All three parsers returned the same valid treee for the sentence. This means that the difference in this case is not in the result but in the computational work needed to get it. For this we can take a look at the number of explored edges.

| Parser | # Explored Edges |
| ----------- | ----------- |
| BoottomUpChartParser | 48 |
| BottomUpLeftCornerChartParser | 29 |
| LeftCornerChartParser | 22 |

Judging by how many edges are explored we can conclude that the LeftCornerChartParser is the most efficient.

### Key Findings
By comparing the sets of edges between the parsers, we found that 19 additional edges are created by the Bottom-Up parser compared to Bottom-Up Left Corner and 26 more than the Left-Corner chart parser. Left-Corner combines bottom-up evidence, which reflects what has actually been read, with top-down constraints that restrict predictions. This makes it so that it filters out both the speculative predictions generated by the Bottom-Up parser and the locally plausible but globally impossible partial expansions that Bottom-Up Left Corner still allows (such as NP → NP * CC NP or VP → V * NP when the continuation does not match the input). This filtering makes Left-Corner the most efficient strategy, exploring only edges that can contribute to a valid full parse.

### Conclusions
This lab showed that
* All three parsers successfully generate the same parse tree, but differ in efficiency.
* Bottom-Up parsing explores the largest number of edges because it predicts every possible rule at every position, regardless of global structure.
* Bottom-Up Left-Corner improves efficiency by restricting predictions based on valid left corners (only symbols that can appear at beginning).
* Left-Corner parsing is the most efficient, exploring less than half as many edges than Bottom-Up.

The combination of top-down and bottom-up constraints prevents both irrelevant predictions and locally plausible but globally impossible partial parses, making it the most efficient out of the three.